In [ ]:
import numpy as np
from numba import jit
import sympy

# Item XV

Considering the following inner product:
$$
\langle p(x),q(x) \rangle =\int_{-1}^{1} \overline{p(x)}q(x) dx
$$

* Let $A= [1|x|x^2|...|x^{n-1}]$ be the "matrix" whose "columns" are the monomials $x^j$, for $j=0,...,n-1$. Each column is a function in $L^2[-1,1]$. compute the $QR$ decomposition of $A$.
* Let $A=[1|\sin(2\pi x)|\sin(4\pi x)|...|x^{n-1}]$ be the "matrix" whose "columns" are the functions $1$ and $\sin(2\pi x)$, for $j=1,...,n-1$. Each column is a function in $L^2[-1,1]$. Compute the $QR$ decomposition of $A$. 
* Do part (a) numerically. Make sure you understand what you are doing since this is a important concept that links symbolic computing with numerical computing.

---

In [ ]:
# This is a generic version of Gram-Schmidt, by default it works on matrices.
# For other uses, replace default argument functions.
def generic_gs(
        elems,
        scalar = lambda a,x : a*x,
        prod = lambda x,y : np.sum(x*y),
        neg = lambda x,y : x-y,
    ):
    """
    elems = [T]
    scalar :: T -> Float -> T
    prod :: T -> T -> Float
    neg :: T -> T -> T
    NOTE: if is used for a regular matrix, elems must be row-wise.
    """
    n = len(elems)
    r = np.zeros((n,n))
    for i in range(n):
        for j in range(i):
            projection = prod(elems[j],elems[i])/prod(elems[j],elems[j])
            r[j,i] = projection
            elems[i] = neg(elems[i],scalar(projection,elems[j]))
        norm2 = prod(elems[i],elems[i])
        if norm2<0:
            print("Warning: negative norm2=%f at i=%d!"%(norm2,i))
        norm = norm2**0.5
        r[i,i] = norm
        elems[i] = scalar(norm**-1,elems[i])
    return r

In [ ]:
def symbolic_inner_product(f,q):
    x = sympy.Symbol('x')
    v = sympy.integrate(f*q,(x,-1,1))
    # We evaluate the expresion as a number because we can't afford the whole symbolic
    # expression...
    return float(v)

In [ ]:
# We define the list of functions for part a
def part_a_funcs(n):
    x = sympy.Symbol('x')
    part = [x**i for i in range(n)]
    return part

In [ ]:
def part_b_funcs(n):
    x = sympy.Symbol('x')
    part = [x**0] + [sympy.sin(2*i*sympy.pi*x) for i in range(1,n)]
    return part

In [ ]:
# We can print an array of functions:
print(part_a_funcs(10))
print(part_b_funcs(10))

In [ ]:
# We compute the decompositions
FUNCS = [part_a_funcs(5),part_b_funcs(5),part_a_funcs(1000),part_b_funcs(100)]
for funcs in FUNCS:
    # Print Original matrix
    print("-"*20+"Functions:")
    print(funcs)
    # Perform QR decomposition using generic G-S
    r = generic_gs(funcs,prod=symbolic_inner_product)
    # Print Q
    print("Q:")
    print(funcs)
    # Print R
    print("R:")
    print(r)

We can see that the functions of the second item are ortogonal, so the $QR$ decomposition gives the indentity (besides the first function $y(x)=1$ that has to be normalized).

At $i=27$ the function coeficients become too small to handle. The norm (inner product with itself) of the functions after substracting the projections becomes small, and negative.

In [ ]:
FUNCS = [part_a_funcs(1000),part_b_funcs(1000)]
for funcs in FUNCS:
    # Print Original matrix
    print("-"*20+"Functions:")
    print(funcs)
    # Perform QR decomposition using generic G-S
    r = generic_gs(funcs,prod=symbolic_inner_product)
    # Print Q
    print("Q:")
    print(funcs)
    # Print R
    print("R:")
    print(r)

---
To do it numerically, let's define a polynomial
$$
p(x) = \sum_{i=0}^{n-1} p_i x^i
$$
as the array of the $p_i$'s.

Then the multiplication becomes:
$$
p(x)q(x) =  \sum_{i=0}^{2n-2} \left( \sum_{k=0}^{i} p_k q_{i-k} \right) x^{i} \,,
$$
then the inner product becomes:
\begin{align*}
\int_{-1}^{1} p(x)q(x) \, dx &= \sum_{i=0}^{2n-2} \left( \sum_{k=0}^{i} p_k q_{i-k} \right) \frac{x^{i+1}}{i+1} |_{x=-1}^{1}
\\ &= \sum_{i=0}^{2n-2} [i \, \text{mod} \, 2= 0] \left( \sum_{k=0}^{i} p_k q_{i-k} \right) \frac{2}{i+1}
\end{align*}

In [ ]:
@jit(nopython=True)
def poly_mult(a,b):
    assert(len(a)==len(b))
    n = len(a)
    total = 0
    for i in range(0,2*n-1,2):
        term = 0
        for k in range(0,i+1):
            if k>=0 and i-k>=0 and k<n and i-k<n:
                term += a[k]*b[i-k]
        term *= 2.0/(i+1.0)
        total += term
    return total

In [ ]:
def poly_print(poly):
    stri = []
    if len(poly)>0 and poly[0] != 0: stri.append("%.3f"%poly[0])
    if len(poly)>1 and poly[1] != 0: stri.append("%.3fx"%poly[1])
    for i in range(2,len(poly)):
        if poly[i]!=0:
            stri.append("%+.3fx%d"%(poly[i],i))
    if len(stri)==0: return "0"
    return " ".join(stri)

def poly_matrix_print(polys,limit=10):
    print("[")
    if len(polys)>2*limit:
        for poly in polys[:limit]:
            print("  "+poly_print(poly)+" |")
        print("  ...")
        for poly in polys[-limit:]:
            print("  "+poly_print(poly)+" |")
    else:
        for poly in polys:
            print("  "+poly_print(poly)+" |")
    print("]")

In [ ]:
poly_mult([1,2,3],[2,5,1])

In [ ]:
for N in (5,10,1000):
    # Print Original matrix
    polys = np.eye(N)
    print("-"*20+" Original (N=%d):"%N)
    poly_matrix_print(polys)
    # Perform QR decomposition using generic G-S
    r = generic_gs(polys,prod=poly_mult)
    # Print Q
    print("Q:")
    poly_matrix_print(polys)
    # Assert that Q is orthonormal
    for i in range(N):
        for j in range(N):
            if i==j:
                assert(np.abs(poly_mult(polys[i],polys[j])-1)<1e-5)
            else:
                assert(np.abs(poly_mult(polys[i],polys[j]))<1e-5)
    # Print R
    print("R:")
    print(r)

We can see that the numerical method fails for $N=1000$. After a closer inspection, this was because, after substracting the projection with the previous functions, the remaining polynomial had very small coefficients, around $i=27$ too.